# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [4]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
full_data_rows_list = []

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)     
        for line in csvreader:
            full_data_rows_list.append(line)
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [7]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

267


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [94]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [95]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )
except Exception as e:
    print(e)

#### Set Keyspace

In [96]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [97]:
query = "CREATE TABLE IF NOT EXISTS songplays"
query = query + "(artist text, first_name text, gender text, item_in_session int, \
last_name text, length float, level text, location text, session_id int, song text, user_id int, \
PRIMARY KEY (item_in_session, session_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [98]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplays (artist, first_name, gender, item_in_session, last_name, length, \
         level, location, session_id, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [99]:
query = "select * from songplays where item_in_session = 4 AND session_id = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.artist, row.song, row.length])

df = pd.DataFrame(data, columns=['artist', 'song', 'lenght'])
df.style.hide_index()

artist,song,lenght


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [72]:
query = "CREATE TABLE IF NOT EXISTS songplays1"
query = query + "(artist text, first_name text, gender text, item_in_session int, \
last_name text, length float, level text, location text, session_id int, song text, user_id int, \
PRIMARY KEY (session_id, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


artist,song,first_name,last_name


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplays (artist, first_name, gender, item_in_session, last_name, length, \
         level, location, session_id, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

In [75]:
query = "select * from songplays where session_id = 182 and user_id = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.artist, row.song, row.first_name, row.last_name])

df = pd.DataFrame(data, columns=['artist', 'song', 'first_name', 'last_name'])
df.style.hide_index()

Error from server: code=2200 [Invalid query] message="unconfigured table songplays"


artist,song,first_name,last_name


## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
query = "CREATE TABLE IF NOT EXISTS songplays2"
query = query + "(artist text, first_name text, gender text, item_in_session int, \
last_name text, length float, level text, location text, session_id int, song text, user_id int, \
PRIMARY KEY (session_id, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplays (artist, first_name, gender, item_in_session, last_name, length, \
         level, location, session_id, song, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

### Drop the tables before closing out the sessions

In [88]:
query = "DROP TABLE IF EXISTS songplays"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS songplays1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songplays2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [89]:
session.shutdown()
cluster.shutdown()